In [1]:
# Setup: versi, path, dan import dasar
import sys, os
print("Python:", sys.version)

# Path dataset
BASE_DIR = r"D:\SKRIPSI\DETEKSI TBC"
DATA_DIR = os.path.join(BASE_DIR, "DATASET")
print("DATA_DIR:", DATA_DIR)

# Install dependensi di dalam notebook (aman di Jupyter)
try:
	import tensorflow as tf
except ModuleNotFoundError:
	import sys
	!{sys.executable} -m pip install tensorflow
	import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras import layers, models, optimizers

print("TensorFlow:", tf.__version__)



Python: 3.13.7 (tags/v3.13.7:bcee1c3, Aug 14 2025, 14:15:11) [MSC v.1944 64 bit (AMD64)]
DATA_DIR: D:\SKRIPSI\DETEKSI TBC\DATASET
TensorFlow: 2.20.0


In [2]:
# Parameter data & split
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 42
VAL_SPLIT = 0.2

train_datagen = ImageDataGenerator(
	preprocessing_function=preprocess_input,
	rotation_range=10,
	width_shift_range=0.05,
	height_shift_range=0.05,
	zoom_range=0.1,
	horizontal_flip=True,
	fill_mode='nearest',
	validation_split=VAL_SPLIT,
)

val_datagen = ImageDataGenerator(
	preprocessing_function=preprocess_input,
	validation_split=VAL_SPLIT,
)

train_gen = train_datagen.flow_from_directory(
	DATA_DIR,
	target_size=IMG_SIZE,
	batch_size=BATCH_SIZE,
	class_mode='binary',
	subset='training',
	seed=SEED,
)

val_gen = val_datagen.flow_from_directory(
	DATA_DIR,
	target_size=IMG_SIZE,
	batch_size=BATCH_SIZE,
	class_mode='binary',
	subset='validation',
	seed=SEED,
)

print("Classes:", train_gen.class_indices)
steps_per_epoch = train_gen.samples // BATCH_SIZE
validation_steps = val_gen.samples // BATCH_SIZE
print("steps_per_epoch:", steps_per_epoch, "validation_steps:", validation_steps)



Found 3035 images belonging to 2 classes.
Found 758 images belonging to 2 classes.
Classes: {'normal': 0, 'tuberculosis': 1}
steps_per_epoch: 94 validation_steps: 23


In [4]:
# Bangun model MobileNetV2 (transfer learning)
base_model = MobileNetV2(
	input_shape=IMG_SIZE + (3,),
	include_top=False,
	weights='imagenet'
)
base_model.trainable = False  # freeze awal

inputs = layers.Input(shape=IMG_SIZE + (3,))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = models.Model(inputs, outputs)

model.compile(
	optimizer=optimizers.Adam(learning_rate=1e-3),
	loss='binary_crossentropy',
	metrics=['accuracy']
)
model.summary()



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [7]:
# Training dengan callback
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

EPOCHS = 20
ckpt_path = os.path.join(BASE_DIR, 'checkpoints', 'mobilenetv2_tbc_best.keras')
os.makedirs(os.path.dirname(ckpt_path), exist_ok=True)

callbacks = [
	EarlyStopping(monitor='val_accuracy', patience=5, mode='max', restore_best_weights=True),
	ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6),
	ModelCheckpoint(ckpt_path, monitor='val_accuracy', mode='max', save_best_only=True)
]

history = model.fit(
	train_gen,
	steps_per_epoch=steps_per_epoch,
	epochs=EPOCHS,
	validation_data=val_gen,
	validation_steps=validation_steps,
	callbacks=callbacks
)



Epoch 1/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 82s 869ms/step - accuracy: 0.9464 - loss: 0.1480 - val_accuracy: 0.8519 - val_loss: 0.3364 - learning_rate: 3.1250e-05
Epoch 2/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 106ms/step - accuracy: 0.9688 - loss: 0.0799 - val_accuracy: 0.8465 - val_loss: 0.3452 - learning_rate: 3.1250e-05
Epoch 3/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 76s 807ms/step - accuracy: 0.9461 - loss: 0.1405 - val_accuracy: 0.8424 - val_loss: 0.3540 - learning_rate: 3.1250e-05
Epoch 4/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 11s 112ms/step - accuracy: 1.0000 - loss: 0.0527 - val_accuracy: 0.8410 - val_loss: 0.3546 - learning_rate: 1.5625e-05
Epoch 5/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 76s 808ms/step - accuracy: 0.9441 - loss: 0.1466 - val_accuracy: 0.8424 - val_loss: 0.3570 - learning_rate: 1.5625e-05
Epoch 6/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 13s 130ms/step - accuracy: 0.9375 - loss: 0.1538 - val_accuracy: 0.8438 - val_loss: 0.3515 - learning_rate: 7.8125e-06


In [8]:
# Evaluasi dan metrik
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve

# Prediksi pada seluruh validation set
val_gen.reset()
y_true = val_gen.classes
# jumlah langkah untuk cover semua sample val
val_steps_full = int(np.ceil(val_gen.samples / BATCH_SIZE))
preds = model.predict(val_gen, steps=val_steps_full)
y_pred_prob = preds.ravel()
y_pred = (y_pred_prob >= 0.5).astype(int)

print("Akurasi:", (y_pred == y_true).mean())
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("Classification Report:\n", classification_report(y_true, y_pred, target_names=list(train_gen.class_indices.keys())))
try:
	auc = roc_auc_score(y_true, y_pred_prob)
	print("ROC-AUC:", auc)
except Exception as e:
	print("ROC-AUC gagal dihitung:", e)



24/24 ━━━━━━━━━━━━━━━━━━━━ 21s 811ms/step
Akurasi: 0.7902374670184696
Confusion Matrix:
 [[596  22]
 [137   3]]
Classification Report:
               precision    recall  f1-score   support

      normal       0.81      0.96      0.88       618
tuberculosis       0.12      0.02      0.04       140

    accuracy                           0.79       758
   macro avg       0.47      0.49      0.46       758
weighted avg       0.69      0.79      0.73       758

ROC-AUC: 0.49588534442903376


In [9]:
# Simpan model dan label map + contoh inferensi
LABEL_MAP = {v: k for k, v in train_gen.class_indices.items()}
print(LABEL_MAP)

export_dir = os.path.join(BASE_DIR, 'exports')
os.makedirs(export_dir, exist_ok=True)
model_path = os.path.join(export_dir, 'mobilenetv2_tbc.keras')
labels_path = os.path.join(export_dir, 'labels.json')

model.save(model_path)
import json
with open(labels_path, 'w') as f:
	json.dump(LABEL_MAP, f)

# Contoh inferensi pada satu gambar
import numpy as np
from tensorflow.keras.preprocessing import image

# ganti ke path gambar uji yang Anda inginkan
sample_path = os.path.join(DATA_DIR, 'tuberculosis', os.listdir(os.path.join(DATA_DIR, 'tuberculosis'))[0])
img = image.load_img(sample_path, target_size=IMG_SIZE)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
prob = model.predict(x)[0,0]
cls = 1 if prob >= 0.5 else 0
print("Prediksi:", LABEL_MAP[cls], "Prob:", float(prob))



{0: 'normal', 1: 'tuberculosis'}
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Prediksi: tuberculosis Prob: 0.5062216520309448


In [10]:
# Fine-tuning: unfreeze sebagian layer MobileNetV2 dan latih lanjut
FINE_TUNE_AT = 120  # unfreeze dari layer ke-120 ke atas (sesuaikan)
base_model.trainable = True
for i, layer in enumerate(base_model.layers):
	layer.trainable = (i >= FINE_TUNE_AT)

model.compile(
	optimizer=optimizers.Adam(learning_rate=1e-4),
	loss='binary_crossentropy',
	metrics=['accuracy']
)
model.summary()

FINE_EPOCHS = 10
ft_ckpt_path = os.path.join(BASE_DIR, 'checkpoints', 'mobilenetv2_tbc_finetuned_best.keras')
callbacks_ft = [
	EarlyStopping(monitor='val_accuracy', patience=4, mode='max', restore_best_weights=True),
	ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6),
	ModelCheckpoint(ft_ckpt_path, monitor='val_accuracy', mode='max', save_best_only=True)
]

fine_tune_history = model.fit(
	train_gen,
	steps_per_epoch=steps_per_epoch,
	epochs=FINE_EPOCHS,
	validation_data=val_gen,
	validation_steps=validation_steps,
	callbacks=callbacks_ft
)



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,626,177 (6.20 MB)

 Non-trainable params: 633,088 (2.42 MB)

Epoch 1/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 98s 921ms/step - accuracy: 0.9407 - loss: 0.1436 - val_accuracy: 0.1929 - val_loss: 5.5426 - learning_rate: 1.0000e-04
Epoch 2/10
 1/94 ━━━━━━━━━━━━━━━━━━━━ 48s 524ms/step - accuracy: 0.9688 - loss: 0.0703

C:\Users\ahuma\AppData\Roaming\Python\Python313\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


94/94 ━━━━━━━━━━━━━━━━━━━━ 11s 109ms/step - accuracy: 0.9688 - loss: 0.0703 - val_accuracy: 0.2024 - val_loss: 5.4189 - learning_rate: 1.0000e-04
Epoch 3/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 75s 800ms/step - accuracy: 0.9790 - loss: 0.0613 - val_accuracy: 0.2826 - val_loss: 3.9050 - learning_rate: 1.0000e-04
Epoch 4/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 11s 110ms/step - accuracy: 1.0000 - loss: 0.0067 - val_accuracy: 0.2921 - val_loss: 3.7166 - learning_rate: 1.0000e-04
Epoch 5/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 80s 853ms/step - accuracy: 0.9833 - loss: 0.0423 - val_accuracy: 0.4769 - val_loss: 2.3501 - learning_rate: 1.0000e-04
Epoch 6/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 14s 143ms/step - accuracy: 1.0000 - loss: 0.0240 - val_accuracy: 0.4457 - val_loss: 2.5006 - learning_rate: 1.0000e-04
Epoch 7/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 82s 872ms/step - accuracy: 0.9880 - loss: 0.0328 - val_accuracy: 0.2609 - val_loss: 4.5988 - learning_rate: 1.0000e-04
Epoch 8/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 14s 138ms/step - accuracy: 0.96

In [12]:
# Plot kurva training dan simpan
import matplotlib.pyplot as plt

def plot_and_save(history_obj, title_prefix, out_dir):
	os.makedirs(out_dir, exist_ok=True)
	hist = history_obj.history
	# Accuracy
	plt.figure()
	plt.plot(hist.get('accuracy', []), label='train_acc')
	plt.plot(hist.get('val_accuracy', []), label='val_acc')
	plt.title(f'{title_prefix} Accuracy')
	plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend()
	plt.tight_layout()
	plt.savefig(os.path.join(out_dir, f'{title_prefix}_accuracy.png'), dpi=150)
	plt.close()
	# Loss
	plt.figure()
	plt.plot(hist.get('loss', []), label='train_loss')
	plt.plot(hist.get('val_loss', []), label='val_loss')
	plt.title(f'{title_prefix} Loss')
	plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend()
	plt.tight_layout()
	plt.savefig(os.path.join(out_dir, f'{title_prefix}_loss.png'), dpi=150)
	plt.close()

plots_dir = os.path.join(BASE_DIR, 'exports', 'plots')
plot_and_save(history, 'baseline', plots_dir)
try:
	plot_and_save(fine_tune_history, 'finetune', plots_dir)
except NameError:
	pass
print('Plot tersimpan di:', plots_dir)



Plot tersimpan di: D:\SKRIPSI\DETEKSI TBC\exports\plots


In [13]:
# Simpan Confusion Matrix dan ROC curve sebagai gambar
from sklearn.metrics import ConfusionMatrixDisplay

fig_dir = os.path.join(BASE_DIR, 'exports', 'figures')
os.makedirs(fig_dir, exist_ok=True)

# Confusion Matrix
y_true = val_gen.classes
val_steps_full = int(np.ceil(val_gen.samples / BATCH_SIZE))
preds = model.predict(val_gen, steps=val_steps_full)
y_pred_prob = preds.ravel()
y_pred = (y_pred_prob >= 0.5).astype(int)

cm = confusion_matrix(y_true, y_pred)
labels = list(train_gen.class_indices.keys())
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
plt.figure(figsize=(4,4))
disp.plot(values_format='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.savefig(os.path.join(fig_dir, 'confusion_matrix.png'), dpi=150)
plt.close()

# ROC curve
try:
	fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
	auc = roc_auc_score(y_true, y_pred_prob)
	plt.figure()
	plt.plot(fpr, tpr, label=f'ROC AUC = {auc:.3f}')
	plt.plot([0,1], [0,1], 'k--')
	plt.xlabel('False Positive Rate')
	plt.ylabel('True Positive Rate')
	plt.title('ROC Curve')
	plt.legend()
	plt.tight_layout()
	plt.savefig(os.path.join(fig_dir, 'roc_curve.png'), dpi=150)
	plt.close()
except Exception as e:
	print('ROC gagal disimpan:', e)

print('Figur disimpan di:', fig_dir)



24/24 ━━━━━━━━━━━━━━━━━━━━ 16s 624ms/step
Figur disimpan di: D:\SKRIPSI\DETEKSI TBC\exports\figures


<Figure size 400x400 with 0 Axes>

In [5]:
# Grad-CAM pada satu contoh gambar (self-contained)
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Konfigurasi minimal bila kernel baru
try:
	BASE_DIR
except NameError:
	BASE_DIR = r"D:\SKRIPSI\DETEKSI TBC"
try:
	IMG_SIZE
except NameError:
	IMG_SIZE = (224, 224)
DATA_DIR = os.path.join(BASE_DIR, 'DATASET')

# Pastikan model tersedia
try:
	model
except NameError:
	model_path = os.path.join(BASE_DIR, 'exports', 'mobilenetv2_tbc.keras')
	model = tf.keras.models.load_model(model_path)

# Tentukan path contoh gambar jika belum ada
try:
	sample_path
except NameError:
	cls_dir = 'tuberculosis' if os.path.isdir(os.path.join(DATA_DIR, 'tuberculosis')) else 'normal'
	sample_path = os.path.join(DATA_DIR, cls_dir, os.listdir(os.path.join(DATA_DIR, cls_dir))[0])

last_conv_layer_name = 'Conv_1'  # terakhir conv MobileNetV2

# Ambil satu sample
img = image.load_img(sample_path, target_size=IMG_SIZE)
x = image.img_to_array(img)
x_batch = np.expand_dims(x, axis=0)
x_batch = preprocess_input(x_batch)

# Temukan base model (MobileNetV2) yang ter-embed di dalam model
base_in_graph = None
for lyr in model.layers:
	# layer model fungsional (nested model) akan punya atribut .layers
	if hasattr(lyr, 'layers') and len(getattr(lyr, 'layers', [])) > 0:
		base_in_graph = lyr
		break

if base_in_graph is None:
	raise ValueError('Base feature extractor (MobileNetV2) tidak ditemukan di dalam model.')

# Ambil layer konvolusi terakhir dari base model ter-embed
try:
	last_conv_layer = base_in_graph.get_layer(last_conv_layer_name)
except ValueError:
	last_conv_layer_name = 'out_relu'
	last_conv_layer = base_in_graph.get_layer(last_conv_layer_name)

# Bangun model fungsional baru: input = input base, output = [feat target, pred akhir]
base_input = base_in_graph.input
conv_target = last_conv_layer.output
x_head = base_in_graph.output
# Rekonstruksi head persis sesuai urutan setelah base model
start_idx = [i for i, lyr in enumerate(model.layers) if lyr is base_in_graph][0] + 1
for lyr in model.layers[start_idx:]:
	x_head = lyr(x_head)

grad_model = tf.keras.models.Model(inputs=base_input, outputs=[conv_target, x_head])

with tf.GradientTape() as tape:
	conv_outputs, predictions = grad_model(x_batch, training=False)
	# Untuk binary sigmoid (1 unit), indeks 0 adalah probabilitas kelas positif
	loss = predictions[:, 0]

grads = tape.gradient(loss, conv_outputs)
pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
conv_outputs = conv_outputs[0]
heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_outputs), axis=-1)
heatmap = np.maximum(heatmap, 0) / (np.max(heatmap) + 1e-8)

# Overlay heatmap ke gambar asli
import cv2
orig = cv2.cvtColor(cv2.imread(sample_path), cv2.COLOR_BGR2RGB)
orig = cv2.resize(orig, IMG_SIZE)
heatmap_resized = cv2.resize(heatmap, IMG_SIZE)
heatmap_color = cv2.applyColorMap(np.uint8(255 * heatmap_resized), cv2.COLORMAP_JET)
heatmap_color = cv2.cvtColor(heatmap_color, cv2.COLOR_BGR2RGB)
overlay = np.uint8(0.4 * heatmap_color + 0.6 * orig)

vis_dir = os.path.join(BASE_DIR, 'exports', 'visualizations')
os.makedirs(vis_dir, exist_ok=True)
vis_path = os.path.join(vis_dir, 'gradcam_sample.png')
cv2.imwrite(vis_path, cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))
print('Grad-CAM disimpan di:', vis_path)



Grad-CAM disimpan di: D:\SKRIPSI\DETEKSI TBC\exports\visualizations\gradcam_sample.png
